In [1]:
import numpy as np
import socket
import threading
import time
import pickle
import random
import matplotlib.pyplot as plt
from networkx import nx
import math as m

In [2]:
class eNodeB(threading.Thread):
    
    
    
    def __init__(self,id,location,n):
        
        threading.Thread.__init__(self)
        self.id=id
        self.loc=location
        self.end=0
        
        self.pan=[]#position announcement az tarafe user ha
        self.uloc=[]#loc user be mme
        
        self.ss_to_send=[]#ferestadan session be sgw
        
        self.ss_to_ack=[]#ferestadan ack session be sgw
        self.ack_to_r=[]#session be girande
        
        self.ack_to_s=[]#ack session be ferestande
        
        self.users=list(range(n))
        
        self.ebuff=[]#ask to send bufferd data
        
        self.ur=[0]*n#hengami ke user registration 1 ra migirim khoone user khas 1 mishe
        
        self.data_for_send=[]# data baraye ferestadan az tarafe ferestande
        
        self.buffd=[]# data az sgw omade,buffer mikonim
        
        self.gbuffy=[]#data bufferi ke bayad vase enod jadid befrestim
        
        
        self.inpos=[0]*n#karbar dar mahdoode enodB hast ya na
        
        self.po=[]
        
        self.p_sgw=8500+self.id
        self.p_mme=7500+self.id
    
        self.enod_sgw_c=1
        self.enod_mme_c=1
        
        
    def l_sgw(self):
        
        
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.bind(('127.0.0.1',8000+self.id))
        sock.listen(10)
        
            
        while True:
            connection, client_address = sock.accept()
                
            while True:
                
                
                data1 = connection.recv(1024);
                
                if data1 :
                            
                    kc=pickle.loads(data1)
                      
                    if kc['type']=='Create Session':
                        
                        self.ack_to_r.append((kc['value'][0],kc['value'][1]))
                        
                  
                    if kc['type']=='Create Session Ack':
                        
                        self.ack_to_s.append((kc['value'][0],kc['value'][1]))                  

                  
                    if kc['type']=='Data Carrier':
                        
                        self.buffd.append(kc['value'])
                       # if self.id==3:
                        #    print(self.buffd)
                        
                    if kc['type']=='Send Me Buffered data':
                        
                        kk=[(item,kc['value'][2]) for item in self.buffd if item[1]==kc['value'][1]]
                        
                        if kk:
                                
                            for i in range(len(kk)):
                                
                                self.gbuffy.append(kk[i])
            
                    if kc['type']=='Buffered data':
                        
                        self.buffd.append(kc['value'][0])
                        
                        
                    if kc['type']=='Handover complete':
                        
                        self.po.append(kc['value'])
                                  
   
                if not data1:
                            
                      
                    break;
         
        return        
        
        
    def l_mme(self):
        
        
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.bind(('127.0.0.1',7000+self.id))
        sock.listen(10)
           
            
        while True:
            
            
            
            
            connection, client_address = sock.accept()
                
            while True:
                
                
                data1 = connection.recv(1024);
                
                
                if data1 :
                            
                    kc=pickle.loads(data1)
                  
                    if kc['type']=='User Registration1':
                        
                        self.ur[kc['value'][0]]=1
                        self.inpos[kc['value'][0]]=1
                        
                        if kc['value'][1]!=-10:
                            
                            self.ebuff.append((kc['value'][1],kc['value'][0]))
                            
                        
                    if kc['type']=='User deregistration':
                        
                        self.inpos[kc['value']]=0
                        
                                    

                if not data1:
                      
                    break;
                
        return        
        
        
    def l_data(self):
        
        
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.bind(('127.0.0.1',6000+self.id))
        sock.listen(10)
        
        
            
            
        while True:
            
            connection, client_address = sock.accept()
                
            while True:
                      
                
                
                data1 = connection.recv(1024);
                
                
                if data1 :
                          
                        
                    kc=pickle.loads(data1)
                  
                    if kc['type']=='Create Session':
                        
                  
                        self.ss_to_send.append((kc['value'][0],kc['value'][1]))
                    
                  
                    if kc['type']=='Create Session Ack':
                        
                        self.ss_to_ack.append((kc['value'][0],kc['value'][1]))
                        
                  
                    if kc['type']=='Data Carrier':
                        
                        self.data_for_send.append(kc['value'])
                                         
   
                if not data1:
        
                      
                    break;
                
        return


    def l_sig(self):
        
        
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.bind(('127.0.0.1',5000+self.id))
        sock.listen(10)
        
        while True:
            
            
            
            connection, client_address = sock.accept()
                
            while True:
                
                      
                
                
                data1 = connection.recv(1024);
                
                
                if data1 :
                          
                        
                    kc=pickle.loads(data1)
                    
                    
                    if kc['type']=='Position announcement':
                        
                        self.pan.append(kc['value'])
                        
                    if kc['type']=='My Location':
                        
                        d=m.sqrt((self.loc[0]-kc['value'][1][0])**2+((self.loc[1]-kc['value'][1][1])**2))
                               
                        self.uloc.append((self.id,kc['value'][0],d))      
                               
              
                                
                    
   
                if not data1:
                      
                    break;
                
                
                  
                
        return
 
    def send(self):
        
        
        
    
        while True:
            
                
            
            if self.enod_sgw_c==1:
                
                self.enod_sgw_c=0
                      
                
                
                    
                
                
                data={};
                
                data['type']="eNodeB-SGW connection"
                data['value']=self.id
                x=pickle.dumps(data)
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.connect(('127.0.0.1',self.p_sgw))
                s.sendall(x)    
                s.close()
        
        
                
            if self.enod_mme_c==1:
                
                
                self.enod_mme_c=0
                      
                
                
                data={};
                
                data['type']="eNodeB-MME connection"
                data['value']=self.id
                x=pickle.dumps(data)
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.connect(('127.0.0.1',self.p_mme))
                s.sendall(x)    
                s.close()
                
            if self.uloc:

                data={};
                
                data['type']='User distance'
                
                data['value']=self.uloc[0]
                self.uloc.pop(0)               
                               
                x=pickle.dumps(data)
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.connect(('127.0.0.1',self.p_mme))
                s.sendall(x)    
                s.close()
                               
            if 1 in self.ur:
                a=self.ur.index(1)
                self.ur[a]=0               
                
                data={};
                
                data['type']='User Registration2'
                
                data['value']=self.id
                              
                               
                x=pickle.dumps(data)
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.connect(('127.0.0.1',3000+a))
                s.sendall(x)    
                s.close()                               
                               
                               
            if self.ss_to_send:
                               
                data={};
                
                data['type']='Create Session'
                
                data['value']=self.ss_to_send[0]
                self.ss_to_send.pop(0)               
                              
                               
                x=pickle.dumps(data)
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.connect(('127.0.0.1',8500+self.id))
                s.sendall(x)    
                s.close()                               
                               
                               
            if self.ack_to_r:
                               
               
                
                data={};
                
                data['type']='Create Session'
                
                data['value']=self.ack_to_r[0]
                self.ack_to_r.pop(0)               
                              
                               
                x=pickle.dumps(data)
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.connect(('127.0.0.1',4000+data['value'][1]))
                s.sendall(x)    
                s.close()
                               
                               
            if self.ss_to_ack:
                               
               
                
                data={};
                
                data['type']='Create Session Ack'
                
                data['value']=self.ss_to_ack[0]
                self.ss_to_ack.pop(0)               
                              
                               
                x=pickle.dumps(data)
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.connect(('127.0.0.1',8500+self.id))
                s.sendall(x)    
                s.close()                               
                               
    
        
    

            if self.ack_to_s:
                               
                data={};
                
                data['type']='Create Session Ack'
                
                data['value']=self.ack_to_s[0]
                self.ack_to_s.pop(0)               
                              
                               
                x=pickle.dumps(data)
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.connect(('127.0.0.1',4000+data['value'][0]))
                s.sendall(x)    
                s.close() 
                               

            if self.data_for_send:
                               
               
                
                data={};
                
                data['type']='Data Carrier'
                
                data['value']=self.data_for_send[0]
                self.data_for_send.pop(0)               
                              
                               
                x=pickle.dumps(data)
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.connect(('127.0.0.1',8500+self.id))
                s.sendall(x)    
                s.close()
                               
                               
            if self.ebuff:
                               
               
                
                data={};
                
                data['type']='Send Me Buffered data'
                
                a=self.ebuff[0]               
                data['value']=(a[0],a[1],self.id)
                               
                self.ebuff.pop(0)               
                              
                               
                x=pickle.dumps(data)
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.connect(('127.0.0.1',8500+self.id))
                s.sendall(x)    
                s.close()                               
                               
                        
                               
            if self.gbuffy:
                               
               
                
                data={};
                
                data['type']='Buffered data'
                
                a=self.gbuffy[0]               
                data['value']=self.gbuffy[0]
                               
                self.gbuffy.pop(0)               
                              
                               
                x=pickle.dumps(data)
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.connect(('127.0.0.1',8500+self.id))
                s.sendall(x)    
                s.close()
                if self.gbuffy:
                               
                    if self.gbuffy[0][1]!=a[0][1]:
                        
                        data={};
                
                        data['type']='Handover complete'
                        data['value']=a[1]
                        x=pickle.dumps(data)
                        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                        s.connect(('127.0.0.1',8500+self.id))
                        s.sendall(x)    
                        s.close()
                               
                               
            
            if 1 in self.inpos:
                
                hit = [i for i,value in enumerate(self.inpos) if value == 1]
                
                if hit:
                
                    
                    for t in range(len(hit)):
                        
                        
                        
                        yy=[item for item in self.buffd if item[1]==hit[t]]
                        
                        if yy:
                            
                        
                            
                        
                            for jj in rr:
                            
                                hp=[item for item in yy if jj==item[0]]
                        
                                if hp:    
                            
                                    if len(set(hp))==hp[0][4]:
                                    
                                 
                                        hp.sort(key=lambda x:x[2])
                            
                                        pj = [i for i,value in enumerate(self.buffd) if value[0] ==jj]
                            
                                        self.buffd = [i for j, i in enumerate(self.buffd) if j not in pj]
                                
                                        data={};
                                
                                        data['type']='Data Carrier'
                                        data['value']=hp
                                        hp=[]
                                        x=pickle.dumps(data)
                                        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                                        s.connect(('127.0.0.1',4000+data['value'][0][1]))
                                        s.sendall(x)    
                                        s.close()
                            
                                    
                   
                               
    def run1(self):
                  
        
        t=[]
        
        
        
        t1=threading.Thread(target=self.l_sgw)
        t2=threading.Thread(target=self.send)
        t3=threading.Thread(target=self.l_mme)
        t4=threading.Thread(target=self.l_data)
        t5=threading.Thread(target=self.l_sig)
        t2.start()        
        t1.start()
        t3.start()
        t4.start()
        t5.start()
        t.append(t1)
        t.append(t2)
        t.append(t3)
        t.append(t4)
        t.append(t5)
        
        for n in t:
                  
            
            n.join()

In [3]:
class MME(threading.Thread):
    
    
    def __init__(self,n1,n2):
        
        threading.Thread.__init__(self)
        
        
        self.enodbs=[]
        
        self.n2=n2
        
        self.enbs=list(range(n1))
        
        self.users=list(range(n2))
        
        self.ack_eb=[0]*n2#ack lazem baraye har user az taraf enodb ha
        
        self.dis=[99999]*n2#min distance har user
        
        self.ebu_old=[-10]*n2;
        
        self.ebu_new=[-1]*n2
               
        self.enb_p=[i+7000 for i in self.enbs]


    def l_enodb(self,i):
        
        
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.bind(('127.0.0.1',7500+i))
        sock.listen(10)
        
        
            
            
        while True:
            
            
            
            connection, client_address = sock.accept()
                
            while True:
                      
                
                
                data1 = connection.recv(1024);
                
                
                if data1 :
                          
                        
                    kc=pickle.loads(data1)
                  
                    if kc['type']=="eNodeB-MME connection":
                        
                  
                        self.enodbs.append(kc['value'])
                        
                  
                  
                    if kc['type']=='User distance':
                        
                        self.ack_eb[kc['value'][1]]=self.ack_eb[kc['value'][1]]+1
                        
                        self.dis[kc['value'][1]]=np.minimum(self.dis[kc['value'][1]],kc['value'][2])
                        
                        if kc['value'][2]==np.minimum(self.dis[kc['value'][1]],kc['value'][2]):
                            
                            self.ebu_new[kc['value'][1]]=kc['value'][0]
                            
                            
                
   
                if not data1:
                      
                    break;
                
                
                  
                
        return

    def send(self):
        
        
        
    
        while True:
            
                
            
            if len(self.enbs) in self.ack_eb:
                
                a=self.ack_eb.index(len(self.enbs))
                
                self.ack_eb[a]=0
                self.dis[a]=99999
                
                data={};
                data['type']="Change route"
                data['value']=self.ebu_new[a],self.users[a]
                x=pickle.dumps(data)
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                
                s.connect(('127.0.0.1',9090))
                s.sendall(x)    
                s.close()
                
                data={};
                data['type']="User Registration1"
                
                
                data['value']=self.users[a],self.ebu_old[a]
                x=pickle.dumps(data)
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                
                s.connect(('127.0.0.1',7000+self.ebu_new[a]))
                s.sendall(x)    
                s.close()
                


                
                if self.ebu_old[a]!=-10:
                    
                    data={};
                    data['type']="User deregistration"
                    data['value']=self.users[a]
                    x=pickle.dumps(data)
                    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                
                    s.connect(('127.0.0.1',7000+self.ebu_old[a]))
                    s.sendall(x)    
                    s.close()
                self.ebu_old[a]=self.ebu_new[a]
                    

    
        return
    

    def run1(self):
                  
        
        t=[]
        
        for j in range(len(self.enbs)):
            
            
            t1=threading.Thread(target=self.l_enodb, args=(j,))
            
                    
            t1.start()
            t.append(t1)            
        
        
        t2=threading.Thread(target=self.send)
        t2.start()      
        t.append(t2)
    
        for n in t:
                  
            
            n.join()    

In [4]:
class SGW(threading.Thread):
    
    
    def __init__(self,n1,n2):
        
        threading.Thread.__init__(self)
        
        self.enodbs=[]
        
        self.enbs=list(range(n1))
        
        self.n1=n1
        
        
        self.ebu=[-5]*n2#enod har user
        
        
        
        self.ii=[]#session be enod
        
        self.jj=[]#ack session be enod
        
        self.data_for_send=[];#ferestadan data
        
        self.gbuff=[]#gather buffer
        
        self.users=list(range(n2))
        
        self.bfs=[]#bufferi ke omade ro befrestam jayi ke bayad
        
        self.hfs=[]#handover ke omade ro befrestam jayi ke bayad
        self.enb_p=[i+8000 for i in self.enbs]
        
    def l_mme(self):
        
        
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.bind(('127.0.0.1',9090))
        sock.listen(10)
        

        while True:
            
            
            
            connection, client_address = sock.accept()
                
            while True:
                      
                
                
                data1 = connection.recv(1024);
                
                
                if data1 :
                          
                        
                    kc=pickle.loads(data1)
                  
                    if kc['type']=='Change route':
                        
                        self.ebu[kc['value'][1]]=kc['value'][0]
                        
                        
                        
   
                if not data1:
                      
                    break;
                
                            
        return

    def l_enodb(self,i):
        
        
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.bind(('127.0.0.1',8500+i))
        sock.listen(10)
        
        
            
            
        while True:
            
            
            
            connection, client_address = sock.accept()
                
            while True:
                      
                
                
                data1 = connection.recv(1024);
                
                
                if data1 :
                          
                        
                    kc=pickle.loads(data1)
                  
                    if kc['type']=="eNodeB-SGW connection":
                  
                        self.enodbs.append(kc['value'])
                        
                  
                  
                    if kc['type']=='Create Session':
                        

                        self.ii.append((kc['value'][0],kc['value'][1]))
                        

                    if kc['type']=='Create Session Ack':
                        
                        self.jj.append((kc['value'][0],kc['value'][1]))

                  
                  
                    if kc['type']=='Data Carrier':
                        
                        self.data_for_send.append(kc['value'])
                        
                    
                    if kc['type']=='Send Me Buffered data':
                        
                        
                        
                        self.gbuff.append(kc['value'])
                        
                    if kc['type']=='Buffered data':
                        
                        
                        self.bfs.append(kc['value'])
                        
                        
                    if kc['type']=='Handover complete':
                        
                        self.hfs.append(kc['value'])                        
                        
                    

                if not data1:
                      
                    break;
                
                
                  
                
        return

    def send(self):
        
        while True:
            
            if self.bfs:
                       
                data={};
                data['type']="Buffered data"
                data['value']=self.bfs[0]
                self.bfs.pop(0)
                x=pickle.dumps(data)
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.connect(('127.0.0.1',self.enb_p[data['value'][1]]))
                s.sendall(x)    
                s.close()             
            
            if self.hfs:
                      
        
                data={};
                data['type']="Handover complete"
                data['value']=self.hfs[0]
                self.hfs.pop(0)
                x=pickle.dumps(data)
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.connect(('127.0.0.1',self.enb_p[data['value']]))
                s.sendall(x)    
                s.close()             

            if self.gbuff:
                      

                data={};
                
                data['type']="Send Me Buffered data"
                data['value']=self.gbuff[0]
                self.gbuff.pop(0)
                x=pickle.dumps(data)
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.connect(('127.0.0.1',self.enb_p[data['value'][1]]))
                s.sendall(x)    
                s.close()            
                
            
            if self.ii:
                
                h=self.ebu[self.ii[0][1]]
                data={};
                data['type']="Create Session"
                data['value']=self.ii[0]
                self.ii.pop(0)
                x=pickle.dumps(data)
                
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.connect(('127.0.0.1',self.enb_p[h]))
                s.sendall(x)    
                s.close()
        
        
                
            if self.jj:
                      
 
                h=self.ebu[self.jj[0][0]]
                data={};
                
                data['type']="Create Session Ack"
                data['value']=self.jj[0]
                self.jj.pop(0)
                x=pickle.dumps(data)
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.connect(('127.0.0.1',self.enb_p[h]))
                s.sendall(x)    
                s.close()
                
               
            if self.data_for_send:
 
                data={};
                
                data['type']="Data Carrier"
                data['value']=self.data_for_send[0]
                a=self.data_for_send[0]
                self.data_for_send.pop(0)
                x=pickle.dumps(data)
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.connect(('127.0.0.1',self.enb_p[self.ebu[a[1]]]))
                s.sendall(x)    
                s.close()            
                
    
        return
    

    def run1(self):
                  
        
        t=[]
        
        for j in range(self.n1):
            
            
            t1=threading.Thread(target=self.l_enodb, args=(j,))
            
                    
            t1.start()
            t.append(t1)            
        
        t3=threading.Thread(target=self.l_mme)
        t2=threading.Thread(target=self.send)
        

        t2.start()        
        t3.start()
        

        t.append(t3)
        t.append(t2)
        

        
        for n in t:
            
                  
            
            n.join()        

In [5]:
class user_g(threading.Thread):
    
    def __init__(self,id,loc,time,n1):
        
        threading.Thread.__init__(self)
        
        self.id=id
        
        self.loc=loc
        
        self.time=time
        
        self.enodbs=list(range(n1))
        
        self.start=1#start 
        
        self.ack_back=[]#ack ro va nakarde pass mifreste
        
        self.p_sig=[i+5000 for i in self.enodbs]
        
        self.p_data=[i+6000 for i in self.enodbs]
        
  
    def l_sig(self):
        
        
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.bind(('127.0.0.1',3000+self.id))
        sock.listen(10)
        
  
        while True:            
            
            connection, client_address = sock.accept()
                
            while True:
                      
                
                
                data1 = connection.recv(1024);
                
                
                if data1 :
                          
                        
                    kc=pickle.loads(data1)
                  
                    if kc['type']=='User Registration2':
                  
                        self.ceb=kc['value']
                    
   
                if not data1:
                      
                    break;
                
                
                  
                
        return

    def l_data(self):
        
        
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.bind(('127.0.0.1',4000+self.id))
        sock.listen(10)
        
 
            
        while True:
                        
            
            connection, client_address = sock.accept()
                
            while True:
                                     
                
                data1 = connection.recv(1024);
                
                
                if data1 :
                          
                        
                    kc=pickle.loads(data1)
                  
                    if kc['type']=='Create Session':
                        
                        self.ack_back.append((kc['value'][0],kc['value'][1]))
                  
                    if kc['type']=='Data Carrier':
                        
                        print('I am '+str(self.id)+' i receive from '+str(kc['value'][0][0])+'\n')
                        for u in range(len(kc['value'])):
                            
                            print(kc['value'][u][3]+'\n')
                        
                        
   
                if not data1:
                      
                    break;
                
                
                  
                
        return

    def send_data(self):
        
        
        
    
        while True:
            
                
            
            if self.ack_back:
                      

                data={};
                
                data['type']="Create Session Ack"
                data['value']=self.ack_back[0]
                self.ack_back.pop(0)
                x=pickle.dumps(data)
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.connect(('127.0.0.1',6000+self.ceb))
                s.sendall(x)    
                s.close()
            
    
        return
    

    def send_loc1(self):
        

                
        self.start=1
        data={};
                
        data['type']="Position announcement"
                    
        data['value']=self.id,self.loc[0]
            
        x=pickle.dumps(data)                
                
        for i in range(len(self.enodbs)):
                    
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            s.connect(('127.0.0.1',self.p_sig[i]))
            s.sendall(x)    
            s.close()
            if i==len(self.enodbs):
                self.start=1
                   
            
        if self.start==1:
            
            for j in range(len(self.loc)):     
                data={};
                
                data['type']="My Location"
                data['value']=self.id,self.loc[j]
                x=pickle.dumps(data)                
                for i in range(len(self.enodbs)):
                    
                    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                    s.connect(('127.0.0.1',self.p_sig[i]))
                    s.sendall(x)    
                    s.close()                 
                
                if j<len(self.time):
                    
                    time.sleep(self.time[j])
                   
        return    
    

    def run1(self):
                  
        
        t=[]
        t1=threading.Thread(target=self.l_data)
        t2=threading.Thread(target=self.send_data)
        t3=threading.Thread(target=self.l_sig)
        t4=threading.Thread(target=self.send_loc1)
        
        t2.start()        
        t1.start()
        t3.start()
        t4.start()

        t.append(t1)
        t.append(t2)
        t.append(t3)
        t.append(t4)

        
        for n in t:
                  
            
            n.join()        

In [6]:
class user_f(threading.Thread):
    
    def __init__(self,id,loc,time,n,rec,delay,file):
        
        threading.Thread.__init__(self)
        
        self.id=id
        
        self.loc=loc
        
        self.time=time
        
        self.delay=delay
        
        
        file1 = open(file, 'r') 
        lines = file1.readlines()

        for i in range(len(lines)-1):
            lines[i]=lines[i][0:len(lines[i])-1]
        
        self.file=lines
        
        self.enodbs=list(range(n))
        
        self.data=0#start baraye ferestadan data
        
        self.rec=rec#girande
        
        self.sack1=0#register enod
        
        self.sack2=0#register enod
        
        self.start=0#start car
        
        self.p_sig=[i+5000 for i in self.enodbs]
        
        self.p_data=[i+6000 for i in self.enodbs]
        
        
    
        
    def l_sig(self):
        
        
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.bind(('127.0.0.1',3000+self.id))
        sock.listen(10)
            
            
        while True:
            
            
            connection, client_address = sock.accept()
                
            while True:
                              
                data1 = connection.recv(1024);
                
                
                if data1 :
                          
                        
                    kc=pickle.loads(data1)
                  
                    if kc['type']=='User Registration2':
                  
                        self.ceb=kc['value']
                    
                        if self.sack2!=1:
                            
                            self.sack1=1
                                
                    
   
                if not data1:
                      
                    break;
                               
        return

    def l_data(self):
        
        
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.bind(('127.0.0.1',4000+self.id))
        sock.listen(10)
        

            
        while True:
                        
            connection, client_address = sock.accept()
                
            while True:
                                     
                
                data1 = connection.recv(1024);
                
                
                if data1 :
                          
                        
                    kc=pickle.loads(data1)
                  
                    if kc['type']=='Create Session Ack':
                  
                        self.data=1
                  

   
                if not data1:
                      
                    break;
                         
                      
        return

    def send_data(self):
        
        
        kn=0
        while True:
            
                
            
            if self.sack1==1 and self.sack2==0:
                
                self.sack2=1
                                  
                data={};
                
                data['type']="Create Session"
                data['value']=self.id,self.rec
                x=pickle.dumps(data)
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.connect(('127.0.0.1',6000+self.ceb))
                s.sendall(x)    
                s.close()
        
        
            if self.data==1:
                
                
                while True:
                    
                    
                
                    data={};
                    data['type']="Data Carrier"
                    data['value']=(self.id,self.rec,kn,self.file[kn],len(self.file))
                    x=pickle.dumps(data)
                    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                    s.connect(('127.0.0.1',self.p_data[self.ceb]))
                    s.sendall(x)    
                    s.close() 
                    if kn==len(self.file)-1:
                        break                    
                    time.sleep(self.delay[kn])
                    kn=kn+1
                    
                
   
    
        return
    

    def send_loc1(self):
        

        time.sleep(0.3)        
        self.start=1
        data={};
                
        data['type']="Position announcement"
                    
        data['value']=self.id,self.loc[0]
            
        x=pickle.dumps(data)                
                
        for i in range(len(self.enodbs)):
                    
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            s.connect(('127.0.0.1',self.p_sig[i]))
            s.sendall(x)    
            s.close()
            if i==len(self.enodbs):
                self.start=1
                   
            
        if self.start==1:
            
            for j in range(len(self.loc)):
                
                
                data={};
                
                data['type']="My Location"
                    
                data['value']=self.id,self.loc[j]
                x=pickle.dumps(data)                
                
                for i in range(len(self.enodbs)):
                    

                    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                    s.connect(('127.0.0.1',self.p_sig[i]))
                    s.sendall(x)    
                    s.close()                 
                
                if j<len(self.time):
                    
                    time.sleep(self.time[j])
            
        return    
    
    
    def run1(self):
                  
        
        t=[]
        

        t1=threading.Thread(target=self.l_data)
        t2=threading.Thread(target=self.send_data)
        t3=threading.Thread(target=self.l_sig)
        t4=threading.Thread(target=self.send_loc1)
        
        t2.start()        
        t1.start()
        t3.start()
        t4.start()

        t.append(t1)
        t.append(t2)
        t.append(t3)
        t.append(t4)

        
        for n in t:
                  
            
            n.join()        

In [7]:
class Network:
    
    
    def __init__(self,n1,n2,loc):
        
        self.n1=n1
        
        self.n2=n2
        
        self.loceb=loc
        
        self.thread=[]
        
        self.user=[]
        


    def init_network(self):
        
        s1=MME(self.n1,self.n2)
        t1=threading.Thread(target=s1.run1)
        t1.start()
        self.thread.append(t1)
        
        
        s2=SGW(self.n1,self.n2)
        t2=threading.Thread(target=s2.run1)
        t2.start()
        self.thread.append(t2)        
        
        for i in range(self.n1):
            sp=eNodeB(i,self.loceb[i],self.n2)
            t1=threading.Thread(target=sp.run1)
            t1.start()
            self.thread.append(t1)
            

    def add_user(self,id,loc,time,delay):
        
        self.user.append((id,loc,time,delay))


    def connection_request(self, sender_id, receiver_id, file_name):
        
        k1=[(item[0],item[1],item[2],self.n1,receiver_id,item[3]) for item in self.user if sender_id==item[0]]
        
        k2=[(item[0],item[1],item[2],self.n1) for item in self.user if receiver_id==item[0]]
        
        s1=user_f(k1[0][0],k1[0][1],k1[0][2],k1[0][3],k1[0][4],k1[0][5],file_name)
        s2=user_g(k2[0][0],k2[0][1],k2[0][2],k2[0][3])
        
        
        t1=threading.Thread(target=s1.run1)
        t2=threading.Thread(target=s2.run1)
        t2.start()
        t1.start()
        self.thread.append(t2)
        self.thread.append(t1)
        
        
    def run(self):
        
        for n in self.thread:
            
            
            n.join()
        
        
        
        
        

In [8]:
if __name__ == "__main__":
    
    network = Network(4,2,[(1,1),(2,1),(1,2),(2,2)])
    network.init_network()
    
    network.add_user(0,[(0,0),(0,2)],[0.5],[0.1,0.8,0.5,0.5])
    network.add_user(1,[(3,0),(3,2)],[1],0)
    network.add_user(2,[(1,0),(0,1),(3,2)],[0.5,2],[0.1,0.7,0.3,0.4])
    network.connection_request(0,1,'1.txt')
    network.connection_request(2,3,'2.txt')
    
    network.run()

Exception in thread Thread-26:
Traceback (most recent call last):
  File "C:\Users\Ali Mehrabian\Anaconda3\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\Ali Mehrabian\Anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-6110f6e95f60>", line 507, in send
    for jj in rr:
NameError: name 'rr' is not defined

Exception in thread Thread-43:
Traceback (most recent call last):
  File "C:\Users\Ali Mehrabian\Anaconda3\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\Ali Mehrabian\Anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-6110f6e95f60>", line 507, in send
    for jj in rr:
NameError: name 'rr' is not defined



IndexError: list index out of range